In [ ]:
# Useful Links
# 1. https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
# 2. https://machinelearningknowledge.ai/keras-implementation-of-resnet-50-architecture-from-scratch/
# 3. https://towardsdatascience.com/resnets-for-cifar-10-e63e900524e0 (Understanding the resnet architecture for cifar10)


In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
PATH = '/content/drive/MyDrive/PhD Studies/fall21/ADVML/results/skip6/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Load necessary libraries
for i in range(5):
  import os
  import pandas as pd
  import numpy as np
  #import cv2
  import tensorflow as tf
  from tensorflow import keras
  from tensorflow.keras import backend as K
  from tensorflow.keras.optimizers import SGD, Adam
  #from google.colab.patches import cv2_imshow
  from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau, CSVLogger
  from tensorflow.keras.models import Sequential, Model,load_model
  from tensorflow.keras.preprocessing.image import ImageDataGenerator
  from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D,Dropout
  from tensorflow.keras.preprocessing import image
  from tensorflow.keras.initializers import glorot_uniform
  from matplotlib import pyplot as plt
  from tensorflow.keras.utils import to_categorical, plot_model



  def load_dataset(dName="CIFAR10"):
      dataset = None
      num_classes = None
      if dName == "CIFAR10":
          num_classes = 10
          dataset = tf.keras.datasets.cifar10.load_data()
      if dName == "CIFAR100":
          num_classes = 100
          dataset = tf.keras.datasets.cifar100.load_data()
      (X_train, y_train), (X_test, y_test) = dataset
      # Convert target value to categorical values
      # One-hot-encoded target values
      y_train = to_categorical(y_train,num_classes)
      y_test = to_categorical(y_test, num_classes)
      
      return (X_train, y_train),(X_test, y_test)
  def divideDataset(X_train, y_train, X_test,y_test):
      dataLength = X_train.shape[0]
      trainLen=0
      dataTrain = []
      dataTest = []
      percent = 0.2
      while(trainLen<dataLength):
          trainLen = int(X_train.shape[0]*percent)
          #print(X_train[:trainLen].shape)
          train = (X_train[:trainLen],y_train[:trainLen])
          retriveLen = int(X_test.shape[0]*percent)
          test = (X_test[:retriveLen],y_test[:retriveLen])
          #print(tuple(train))
          dataTrain.append(train)
          dataTest.append(test)
          #print("\n")
          percent +=0.2
      return dataTrain, dataTest
  (X_train, y_train),(X_test, y_test) = load_dataset()
  dataTrain, dataTest = divideDataset(X_train, y_train, X_test, y_test) # this contains the list of 5 different datasets
  def normalizeInput(X_train,X_test):
      X_train = X_train.astype('float32')
      X_test = X_test.astype('float32')
      mean = np.mean(X_train, axis=(0,1,2,3))
      std = np.std(X_train,axis=(0,1,2,3))
      X_train = (X_train-mean)/(std+1e-7)
      X_test = (X_test-mean)/(std+1e-7)
      
      return X_train, X_test
  ##### Include Little Data Augmentation 
  '''
  batch_size = 64 # try several values

  train_DataGen = tf.keras.preprocessing.image.ImageDataGenerator(zoom_range=0.2, 
                                                                  width_shift_range=0.1, 
                                                                  height_shift_range = 0.1, 
                                                                  horizontal_flip=True)
  
  valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

  train_set_conv = train_DataGen.flow(X_train, y_train, batch_size=batch_size) # train_lab is categorical 
  valid_set_conv = valid_datagen.flow(X_test, y_test, batch_size=batch_size) # so as valid_lab
  '''
  X_train, X_test = normalizeInput(dataTrain[0][0], dataTest[0][0])
  y_train = dataTrain[0][1]
  y_test = dataTest[0][1]
  # ResNet Implementation
  # Code Link: https://github.com/Gurupradeep/CIFAR-10-Object-Recognition-in-Images/blob/master/Models/ResNet.ipynb
  
  def dataAugmentation():
    datagen = ImageDataGenerator(
          featurewise_center=False,  # set input mean to 0 over the dataset
          samplewise_center=False,  # set each sample mean to 0
          featurewise_std_normalization=False,  # divide inputs by std of the dataset
          samplewise_std_normalization=False,  # divide each input by its std
          zca_whitening=False,  # apply ZCA whitening
          rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
          width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
          height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
          horizontal_flip=False,  # randomly flip images
          vertical_flip=False)  # randomly flip images
    datagen.fit(X_train)
    return datagen
  daug = dataAugmentation()
  
  # RESNET50 WITH SKIP CONNECTION 6: Implemented by Lokesh
  def createResnetWithSkip6(x, f,k, s,p,counter):
    '''
    Argument
    x - input
    f - filter_size
    k - kernel size
    s - stride
    p - padding
    '''
    block_name = 'stage_2'+counter
    x_skip = x
    #print("Recevied Input at Stage ->: "+counter, x.shape)
    layers = len(f)
    for i in range(layers):
      x = Conv2D(f[i],kernel_size=(k[i],k[i]),strides=(s[i],s[i]),padding=p[i])(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
    x = Conv2D(f[layers-1],kernel_size=(k[layers-1],k[layers-1]),strides=(s[layers-1],s[layers-1]),padding=p[layers-1],name=block_name+'b')(x)
    x = BatchNormalization()(x)
    #print(x.shape)

    if x.shape[1] != x_skip.shape[1]:
      x_skip = Conv2D(f[layers-1],kernel_size=(1,1),strides=(2,2),padding='valid')(x_skip)
    else:
      x_skip = Conv2D(f[layers-1],kernel_size=(1,1),strides=(1,1),padding='valid')(x_skip)

    x_skip = Conv2D(f[layers-1],kernel_size=(1,1),strides=(1,1),padding='valid')(x_skip)
    x_skip = BatchNormalization()(x_skip)
    x = Add()([x,x_skip])
    x = Activation('relu')(x)
    #print("Stage 2:", (x_skip.shape,x.shape))

    return x

  def resnet50():
    inputs = Input(shape = (32,32,3), name = "image_input")
    x = ZeroPadding2D((3,3))(inputs)
    # Initial Convolutional block
    
    x = Conv2D(64, kernel_size=(7,7), strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    print("before Max pooling: ", x.shape)
    x = MaxPooling2D((3,3),strides=(2,2))(x)
    x = createResnetWithSkip6(x,[64,64,256,64,64,256],[1,3,1,1,3,1],[1,1,1,1,1,1],['valid','same','valid','valid','same','valid'],'1')
    x = createResnetWithSkip6(x,[64,64,256,128,128,512],[1,3,1,1,3,1],[1,1,1,2,1,1],['valid','same','valid','valid','same','valid'],'2')
    x = createResnetWithSkip6(x,[128,128,512,128,128,512],[1,3,1,1,3,1],[1,1,1,1,1,1],['valid','same','valid','valid','same','valid'],'3')
    x = createResnetWithSkip6(x,[128,128,512,256,256,1024],[1,3,1,1,3,1],[1,1,1,2,1,1],['valid','same','valid','valid','same','valid'],'4')
    x = createResnetWithSkip6(x,[256,256,1024,256,256,1024],[1,3,1,1,3,1],[1,1,1,1,1,1],['valid','same','valid','valid','same','valid'],'5')
    x = createResnetWithSkip6(x,[256,256,1024,256,256,1024],[1,3,1,1,3,1],[1,1,1,1,1,1],['valid','same','valid','valid','same','valid'],'6')
    x = createResnetWithSkip6(x,[256,256,1024,512,512,2048],[1,3,1,1,3,1],[1,1,1,2,1,1],['valid','same','valid','valid','same','valid'],'7')
    x = createResnetWithSkip6(x,[512,512,2048,512,512,2048],[1,3,1,1,3,1],[1,1,1,1,1,1],['valid','same','valid','valid','same','valid'],'8')
    x = AveragePooling2D((2,2),padding='same')(x)
    x = Flatten()(x)
    x = Dense(10,activation='softmax')(x)
    model = Model(inputs=inputs,outputs=x, name='LResNet50')
    #print(x.shape)
    return model

  model = resnet50()
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
  #model.summary()
  
  X_train, X_test = normalizeInput(dataTrain[i][0], dataTest[i][0])
  y_train = dataTrain[i][1]
  y_test = dataTest[i][1]
  epochs = 300
  batch_size = 1024
  earlyStop = EarlyStopping(monitor='loss',patience=5,mode='auto')
  csvLogger = CSVLogger('/content/drive/MyDrive/PhD Studies/fall21/ADVML/results/skip6/'+str(i)+'.csv')
  '''history = model.fit(daug.flow(X_train, y_train, batch_size=batch_size),
                          steps_per_epoch=X_train.shape[0] // batch_size,
                          epochs=epochs,verbose=1,validation_data=(X_test,y_test),callbacks=[csvLogger,earlyStop])'''
  history = model.fit(X_train,y_train, epochs=300,batch_size=1024,validation_data=(X_test,y_test),verbose=1, callbacks=[csvLogger,earlyStop])



  import gc
  del model
  gc.collect()
  K.clear_session()
  tf.compat.v1.reset_default_graph()

before Max pooling:  (None, 16, 16, 64)
Epoch 1/300
9/9 [==============================] - 16s 868ms/step - loss: 3.2466 - accuracy: 0.1330 - val_loss: 2.3098 - val_accuracy: 0.1160
Epoch 2/300
9/9 [==============================] - 6s 650ms/step - loss: 2.1886 - accuracy: 0.2136 - val_loss: 2.2876 - val_accuracy: 0.1370
Epoch 3/300
9/9 [==============================] - 6s 660ms/step - loss: 2.0154 - accuracy: 0.2352 - val_loss: 2.2711 - val_accuracy: 0.1585
Epoch 4/300
9/9 [==============================] - 6s 677ms/step - loss: 1.9594 - accuracy: 0.2637 - val_loss: 2.2651 - val_accuracy: 0.1635
Epoch 5/300
9/9 [==============================] - 6s 669ms/step - loss: 1.9081 - accuracy: 0.2845 - val_loss: 2.2607 - val_accuracy: 0.1570
Epoch 6/300
9/9 [==============================] - 6s 675ms/step - loss: 1.8775 - accuracy: 0.2939 - val_loss: 2.2765 - val_accuracy: 0.1075
Epoch 7/300
9/9 [==============================] - 6s 658ms/step - loss: 1.8205 - accuracy: 0.3174 - val_loss: 2.